In [ ]:
import numpy as np
from numpy import linalg as la
from numpy import *
from numpy import random as rrr
import matplotlib.pyplot as plt
from matplotlib import cm
from tqdm import tqdm
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from time import time
import joblib
from collections import namedtuple, deque
import math
import random as rnd
from scipy import stats
# importing movie py libraries
# from moviepy.editor import VideoClip
# from moviepy.video.io.bindings import mplfig_to_npimage
from fhn_system_2d import simulate_trajectory

import gc

In [ ]:
print (torch.__version__, torch.cuda.is_available())
print(torch.version.cuda)
print (torch.cuda.get_device_name())
#device= 'cpu'
device= 'cuda'

In [ ]:
## Choose a system (this is just for name)
system = 'fhn' 
# system = 'double_well'

In [ ]:
num_steps = 10000
k_grid = 32

In [ ]:
"""
Main script for FirzHugh Nagumo system simulation
"""

# Set parameters for the FHN system 
gamma = 1
beta = 1
delta = 0.25
epsilon = 0.1


a1= 1/3
b1= 0.5
# b2= 0
b2= 0.1

# Define again later
DX= 0.15**2
DY= (epsilon*0.15)**2

# Create a grid of points for phase space visualization
x_plot = np.linspace(-3, 3, 100)
y_plot = np.linspace(-3, 3, 100)
XX, YY = np.meshgrid(x_plot, y_plot)

# Calculate vector field for phase space plot
U = (XX - a1*XX**3-YY )
V = epsilon*(XX+ b1)  # dy/dt at t=0

# Normalize the arrows
magnitude = np.sqrt(U**2 + V**2)
U_norm = U / magnitude
V_norm = V / magnitude

# Plot the phase space
plt.figure(figsize=(6, 4))
plt.streamplot(XX, YY, U, V, density=1.5, color='darkblue', linewidth=0.7)
plt.quiver(XX[::10, ::10], YY[::10, ::10], U_norm[::10, ::10], V_norm[::10, ::10], 
           color='red', scale=25)
plt.title('FirzHugh Nagumo Phase Space')
plt.xlabel('x (position)')
plt.ylabel('y (velocity)')
plt.grid(alpha=0.3)
plt.show()

def get_single_trajectory(x0, y0, T):
    """
    Produce SDE trajectory starting at (x0, y0) over T steps.
    This function delegates the simulation to duffing_system_2d.simulate_trajectory.
    
    Args:
        x0: Initial x-coordinate (position)
        y0: Initial y-coordinate (velocity)
        T: Total simulation time steps
        
    Returns:
        Tuple of (data_matrix_single, lag_time)
    """
    return simulate_trajectory(x0, y0, T, h=1e-4, n_steps=100, beta= beta, delta= delta, 
                        epsilon= epsilon, a1=a1, b1=b1, b2=b2, DX= DX, DY= DY)

In [ ]:
def true_evals (n_max= 3):
    eigs = []
    mu1= 0.05
    # omega1= 0.588
    omega1= 0.185 # 0.20
    # Compute true eigenvalues for the l = 0 case
    for n in range(-n_max, n_max+1):
         # Skip n = 0 case
        lambda_cont = -n**2*mu1  + 1j*n*omega1
        eigs.append(lambda_cont)
    
    return np.array(eigs)

# how many eigenvalues to keep / plot
TOP_K = 2        # Define again later
true_evalues = true_evals(TOP_K)
print('true eigenvalues: ', true_evalues)

# Create scatter plot of eigenvalues in the complex plane
plt.figure()
plt.scatter(true_evalues.real, true_evalues.imag)

# Add reference lines for real and imaginary axes
plt.axhline(0, linewidth=0.5)
plt.axvline(0, linewidth=0.5)

# Set plot limits to focus on non-positive real parts
# plt.xlim(-3, 2)   # Real part range
# plt.ylim(-2, 2)    # Imaginary part range
plt.xlim(-0.8, 0.5)   # Real part range
plt.ylim(-0.5, 0.5)    # Imaginary part range

# Label and title
plt.title("Eigenvalues in the Complex Plane")
plt.xlabel("Real Part")
plt.ylabel("Imaginary Part")

# Equal aspect ratio to preserve geometric interpretation
plt.gca().set_aspect('equal', 'box')

# Display the plot
plt.show()


In [ ]:
reward_hist= joblib.load (f'ppo_history{k_grid}x{k_grid}_{num_steps}steps_example_{system}_final.jbl')
os.makedirs(f'figures_ppo_{system}', exist_ok=True)

In [ ]:
def make_trajectory_from_state(state, chunk_len=100):  # Default is still 100
    state_len = state.shape[1]
    chunk_list = []
    T = 3*chunk_len  # Use the parameter
    
    for ii in arange(state_len):
        x0_single = state[0, ii]
        y0_single = state[1, ii]
        
        data_matrix_single, lag_time = get_single_trajectory(x0_single, y0_single, T)
        chunk_list.append(data_matrix_single)
    
    data_matrix_single = hstack(chunk_list)
    return data_matrix_single, lag_time


rewards_all=[]
states_all= []
actions_all= []
next_states_all= []
for ii in arange(len (reward_hist)):
        curr_list= reward_hist[ii]
        rewards_all.append (curr_list[-1])
        states_all.append (curr_list[0])
        actions_all.append (curr_list[1])
        next_states_all.append (curr_list[2])


def make_trajectory_from_state_list (state_list):
    n_traj= len (state_list)
    traj_list= []
    for kk in tqdm (arange (n_traj)):
        trajectory, lag_time= make_trajectory_from_state(state_list[kk])
        traj_list.append (trajectory)
        trajectory_final= np.hstack (traj_list)
    return trajectory_final, lag_time

In [ ]:
from solver_sdmd_torch_gpu2 import KoopmanNNTorch, KoopmanSolverTorch
"""
Refactored SDMD-PPO script (solver-2 version)
------------------------------------------------
• Same optimisation style as the previous refactor
• All comments are in English
• y-axis for eigenvalue plots fixed at -3…3
• Unit-circle helper included but **NOT** called (uncomment if needed)

Assumed globals already defined elsewhere:
    states_all, system, device,
    make_trajectory_from_state, true_evals,
    KoopmanNNTorch, KoopmanSolverTorch
"""

import os, gc, torch
import numpy as np
import matplotlib.pyplot as plt
from numpy import arange, sign
from tqdm import tqdm


# ─────────────────────────── HELPERS ─────────────────────────── #
def dynamic_params(step_view: int) -> tuple[int, int]:
    """Return (chunk_len, n_traj) based on current step_view."""
    chunk_len = max(10, 20 + step_view // 10)
    if step_view < 100:
        n_traj = 2
    elif step_view < 500:
        n_traj = 3
    elif step_view < 1000:
        n_traj = 5
    else:
        n_traj = 8 + step_view // 1200
    return chunk_len, n_traj


def build_trajectory(state_slice, chunk_len: int):
    """Concatenate trajectories generated from a list of initial states."""
    trajs = []
    for st in tqdm(state_slice, desc="building-traj", leave=False):
        traj, lag = make_trajectory_from_state(st, chunk_len=chunk_len)
        trajs.append(traj)
    return np.hstack(trajs), lag


def viz_multiplier(step_view: int) -> float:
    """Scale factor for larger trajectory used in eigenfunction plots."""
    if step_view < 100:
        return 1.0
    elif step_view < 500:
        return 1.5
    return 2.0


def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)
    return path


def unit_circle(ax):
    """Optional helper to draw a unit circle (currently unused)."""
    theta = np.linspace(0, 2 * np.pi, 200)
    ax.plot(np.cos(theta), np.sin(theta), "--", lw=0.5)

In [ ]:
#### Test 1 ####

# ─────────────────────────── CONFIG ──────────────────────────── #
STEP_VIEWS = [100, 250, 500, 750, 1000, 1500, 2000, 3000, 4000, 5000, 6000, 7000, 8000]
TRAIN_SPLIT = 0.7
EPOCHS = 6
BATCH_SIZE = 256
LR = 1e-5
LR_DECAY = 0.8
REG = 0.1
LAYER_SIZES = [30]
N_PSI_TRAIN = 27
TOP_K = 5        # how many eigenvalues to keep / plot
Y_LIM = (-3, 3)  # fixed y‑axis range

epsilon = 0.01
DX= 0.15**2
DY= (epsilon*0.15)**2

# ─────────────────────────── MAIN LOOP ────────────────────────── #
NUM_ITER = 2
for it in range(NUM_ITER):
    print(f"\n{'='*50}\nStarting iteration {it+1}/{NUM_ITER}\n{'='*50}")
    out_dir = ensure_dir(f"figures_ppo_{system}/iteration_{it+1}")

    X_eval_accum = []  # hold evaluation points across step_views

    for step in STEP_VIEWS:
        chunk_len, n_traj = dynamic_params(step)
        print(f"step={step}  n_traj={n_traj}  chunk_len={chunk_len}")

        # ── build training data ─────────────────────────────────── #
        traj_slice = states_all[step : step + n_traj]
        data_mat, lag = build_trajectory(traj_slice, chunk_len)

        X = data_mat[:, :-1, :].reshape(-1, data_mat.shape[2])
        Y = data_mat[:, 1:, :].reshape(-1, data_mat.shape[2])

        split = int(TRAIN_SPLIT * len(X))
        train = [X[:split], Y[:split]]
        valid = [X[split:], Y[split:]]

        # ── initialise solver ───────────────────────────────────── #
        ckpt        = f"example_{system}_dqn_iter{it+1}_ckpt.torch"
        fnn_ckpt    = f"example_{system}_fnn_dqn_iter{it+1}.torch"
        coeff_file  = f"sde_coefficients_example_{system}_dqn_iter{it+1}.jbl"

        basis = KoopmanNNTorch(2, LAYER_SIZES, N_PSI_TRAIN).to(device)
        solver = KoopmanSolverTorch(
            dic=basis,
            target_dim=X.shape[1],
            reg=REG,
            checkpoint_file=ckpt,
            fnn_checkpoint_file=fnn_ckpt,
            a_b_file=coeff_file,
            generator_batch_size=2,
            fnn_batch_size=32,
            delta_t=lag
        )

        solver.build_with_generator(
            data_train=train,
            data_valid=valid,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            lr=LR,
            log_interval=10,
            lr_decay_factor=LR_DECAY
        )

        # ── eigenvalues ─────────────────────────────────────────── #
        eigs = solver.eigenvalues.T
        eigs_gen = (eigs - 1) / lag
        eigs_sorted = -np.sort_complex(-eigs_gen)

        print(f"   top-{TOP_K} eigs:", eigs_sorted[:TOP_K])

        # ── eigenvalue plot (fixed axes & scale) ───────────────────────── #
        fig, ax = plt.subplots(figsize=(6, 4))
        
        # 1) plot analytic / reference items first  ➜ they never move
        true = true_evals(int((TOP_K - 1) / 2))
        ax.scatter(true.real, true.imag, c="r", marker="x", label="True")
        
        # Optional: uncomment to draw a reference unit circle that never moves
        # theta = np.linspace(0, 2 * np.pi, 200)
        # ax.plot(np.cos(theta), np.sin(theta), "--", lw=0.5, color="gray")
        
        # 2) plot estimated eigenvalues (may lie outside limits — that’s OK)
        lead = eigs_sorted[:TOP_K]
        ax.scatter(lead.real, lead.imag, color="tab:blue", label="Estimated")
        
        # 3) absolutely fix axes & aspect every time
        ax.set_xlim(-2, 1)
        ax.set_ylim(-1, 1)
        ax.set_aspect("equal", adjustable="box")   # keeps circle & grid square
        # ax.set_title(f"Eigenvalues (step={step})", fontsize=18)
        ax.set_xlabel("Real part");  ax.set_ylabel("Imaginary part")
        ax.grid(ls="--", lw=0.5)
        ax.legend(loc="upper left")
        
        fig.tight_layout()
        fig.savefig(f"{out_dir}/eigs_{system}_iter{it+1}_step{step}.png")
        plt.close(fig)

        # # eigenvalue TXT
        # txt = f"{out_dir}/top{TOP_K}_eigs_iter{it+1}_step{step}.txt"
        # with open(txt, "w") as f:
        #     f.write(f"Top {TOP_K} eigenvalues (generator)  iter={it+1}  step={step}\n")
        #     f.write(f"{'i':<3} {'Real':>12} {'Imag':>12} {'|λ|':>12} {'θ°':>10}\n")
        #     for i, ev in enumerate(lead, 1):
        #         f.write(f"{i:<3} {ev.real:12.6f} {ev.imag:12.6f} "
        #                 f"{abs(ev):12.6f} {np.angle(ev, deg=True):10.2f}\n")

        # ── eigenfunction evaluation & plot ─────────────────────── #
        n_big = max(2, int(viz_multiplier(step) * n_traj))
        traj_big, _ = build_trajectory(states_all[step : step + n_big], chunk_len)
        X_eval_accum.append(traj_big.reshape(-1, traj_big.shape[2]))
        X_all = np.vstack(X_eval_accum)

        efuns = solver.eigenfunctions(X_all)
        ref_sign = sign(solver.eigenfunctions(np.array([[0., 1.]]))[0].real)

        # Create single subplot for 2nd eigenfunction
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))
        vals = np.real(efuns) * ref_sign

        # Plot 2nd eigenfunction
        sc = ax.scatter(X_all[:, 0], X_all[:, 1], c=vals[:, 1],
                    cmap="coolwarm", alpha=0.6)
        # ax.set_title(f"2nd Eigenfunction (step={step})", fontsize=18)
        ax.set(xlim=(-3, 3), ylim=(-3, 3))
        cbar = fig.colorbar(sc, ax=ax, shrink=0.8, pad=0.1) 
        cbar.ax.tick_params(labelsize=18, width=2)

        fig.tight_layout()
        fig.savefig(f"{out_dir}/efuns_{system}_iter{it+1}_step{step}.png")
        plt.close(fig)

        # ── memory cleanup ──────────────────────────────────────── #
        del efuns, fig, ax, sc
        if 'cuda' in str(device):
            torch.cuda.empty_cache()
        gc.collect()

    print(f"Iteration_{it+1} finished — figures saved to {out_dir}")

print("\nAll iterations completed!")
